# Bottle Data

### Purpose
This notebook provides a quick outline and example of working with OOI Discrete Summary Spreadsheet bottle data. It includes how to load it, how to parse data quality flags, how to derive some important values, etc.

At each [boiler plate stuff about OOI cruises]

### Data
The cruise data is hosted on OOI's Alfresco document server at https://alfresco.oceanobservatories.org, hereafter reffered to as Alfresco. Cruise data is hosted under OOI > {Array} > Cruise Data > {Cruise}. Cruise data includes the following key pieces of information: Cruise Reports, Ship Data (e.g. sonars, underway instruments, etc.), CTD and Water Sampling Data.

We are principally concerned with the Water Sampling data, which can be found under {Cruise} > Ship Data > Water Sampling. The Water Sampling folder contains both digitized and scanned CTD logs as well as the results from the labs for each .

In [1]:
import os, sys, re, ast
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
sys.path.append("../../OS2022/OS2022/")
from bottle_utils import *

In [4]:
pd.set_option("display.max_columns", None)

---
### Niskin & Discrete Bottle Sampling
These are discrete water samples collected via Niskin Bottle casts during deployment and recovery of the moored instrumentation. The data is downloaded from OOI Alfresco website as excel files. Parameters sampled are:
* Oxygen
* Salinity
* Nutrients: phosphate, nitrate, nitrite, ammonium, silicate
* Chlorophyll
* Carbon system: Dissolved inorganic carbon, total alkalinity, pH

First, we're going to load, clean, and process the discrete water sampling from Irminger Array.

In [9]:
bottleData = pd.DataFrame()
dataDir = "../data/water_sampling/"
for file in os.listdir(dataDir):
    if ("Irminger" in file) and file.endswith(".csv") and ("Discrete" in file):
        fileData = pd.read_csv(dataDir + "/" + file)
        bottleData = bottleData.append(fileData)

In [10]:
bottleData.head()

,Cruise,Station,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth at Start Position [m],CTD File,CTD File Flag,Niskin/Bottle Position,Niskin Flag,CTD Bottle Closure Time [UTC],CTD Pressure [db],CTD Pressure Flag,CTD Depth [m],CTD Latitude [deg],CTD Longitude [deg],CTD Temperature 1 [deg C],CTD Temperature 1 Flag,CTD Temperature 2 [deg C],CTD Temperature 2 Flag,CTD Conductivity 1 [S/m],CTD Conductivity 1 Flag,CTD Conductivity 2 [S/m],CTD Conductivity 2 Flag,CTD Salinity 1 [psu],CTD Salinity 2 [psu],CTD Oxygen [mL/L],CTD Oxygen Flag,CTD Oxygen Saturation [mL/L],CTD Fluorescence [mg/m^3],CTD Fluorescence Flag,CTD Beam Attenuation [1/m],CTD Beam Transmission [%],CTD Transmissometer Flag,CTD pH,CTD pH Flag,Discrete Oxygen [mL/L],Discrete Oxygen Flag,Discrete Oxygen Replicate Flag,Discrete Chlorophyll [ug/L],Discrete Phaeopigment [ug/L],Discrete Fo/Fa Ratio,Discrete Fluorescence Flag,Discrete Fluorescence Replicate Flag,Discrete Phosphate [uM],Discrete Silicate [uM],Discrete Nitrate [uM],Discrete Nitrite [uM],Discrete Ammonium [uM],Discrete Nutrients Flag,Discrete Nutrients Replicate Flag,Discrete Salinity [psu],Discrete Salinity Flag,Discrete Salinity Replicate Flag,Discrete Alkalinity [umol/kg],Discrete Alkalinity Flag,Discrete Alkalinity Replicate Flag,Discrete DIC [umol/kg],Discrete DIC Flag,Discrete DIC Replicate Flag,Discrete pCO2 [uatm],pCO2 Analysis Temp [deg C],Discrete pCO2 Flag,Discrete pCO2 Replicate Flag,Discrete pH [Total scale],pH Analysis Temp [deg C],Discrete pH Flag,Discrete pH Replicate Flag,Calculated Alkalinity [umol/kg],Calculated DIC [umol/kg],Calculated pCO2 [uatm],Calculated pH,Calculated CO2aq [umol/kg],Calculated Bicarb [umol/kg],Calculated CO3 [umol/kg],Calculated Omega-C,Calculated Omega-A
0,AR35-05,1,Acoustic Release,62.767617,-28.647200,2019-08-03T09:15:00.000Z,1,*0000000000000001,1856,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999.0,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
1,AR35-05,2,GI01SUMO,59.939817,-39.522150,2019-08-05T16:21:00.000Z,2,*0000000000000001,2702,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999.0,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
2,AR35-05,3,GI02HYPM,59.969850,-39.508367,2019-08-06T13:20:00.000Z,3,*0000000000000001,2669,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999.0,-9999999,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999.0,-9999999.0,-9999999.0,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999.0,-9999999,-9999999,-9999999,-99999

Check on the available cruises

In [11]:
bottleData["Cruise"].unique()

array(['AR35-05', 'AR7-01', 'AR21', 'KN221-04', 'AT30-01', 'AR30-03'],
      dtype=object)

Replace the fill value of ```-9999999``` with NaNs. Some columns were imported as floats and others as strings or objects, so this needs to be done twice:

In [16]:
bottleData = bottleData.replace(to_replace="-9999999", value=np.nan)
bottleData = bottleData.replace(to_replace=-9999999, value=np.nan)

Convert times from strings to pandas datetime objects and remove the timezone:

In [17]:
bottleData["Start Time [UTC]"] = bottleData["Start Time [UTC]"].apply(lambda x: convert_times(x))
bottleData["CTD Bottle Closure Time [UTC]"] = bottleData["CTD Bottle Closure Time [UTC]"].apply(lambda x: convert_times(x))

Some of the data returned from the labs which process samples contain non-numeric indicators for if data were below detection threshold. These entries typically have a "less than" sign. Convert any values with a "<", which indicates a value not statistically significant from zero, with zero:

In [18]:
bottleData = bottleData.applymap(not_statistically_sigificant)

#### Filter data
Next, we want to filter the data based on the data flag associated with the sample. First, we should drop all of the ```NaN``` values in the Niskin column to eliminate entries without any bottle samples. While these CTD casts may be of interest, the bottle data does not have any data of which we can make use of.

In [19]:
bottleData = bottleData.dropna(subset=["Niskin/Bottle Position"])

#### Interpret Flags
Next, we want to parse the Data Quality Flags in the bottle data. 

We intrept the data into the standard QARTOD flaging scheme of:
* 1 = good
* 2 = not run
* 3 = suspect
* 4 = bad
* 9 = missing

The "Replicate Flags" are reduced into a boolean value indicating that either there is a replicate sample or not. By replicate, we mean if there are more than one sample collected from a depth, e.g. two Niskins were closed at a particular depth and oxygen sampled from both. It does *not* mean that a particular analysis was repeated on the same sample multiple times, e.g. repeating nutrient analysis on a single Niskin to derive a final value.

In [ ]:
#### Interpret Flags
Next, we want to parse the Data Quality Flags in the bottle data. 

We intrept the data into the standard QARTOD flaging scheme of:
* 1 = good
* 2 = not run
* 3 = suspect
* 4 = bad
* 9 = missing

The "Replicate Flags" are reduced into a boolean value indicating that either there is a replicate sample or not. By replicate, we mean if there are more than one sample collected from a depth, e.g. two Niskins were closed at a particular depth and oxygen sampled from both. It does *not* mean that a particular analysis was repeated on the same sample multiple times, e.g. repeating nutrient analysis on a single Niskin to derive a final value.